In [1]:
import orjson
import os 

os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1, 2, 3" 

path = './11.03.json'

with open(path, "rb") as f:
    json_data = orjson.loads(f.read())

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer

galactica_6_7b_path = "/home/jihuawei/jhw2023fall/eval_assumption/galactica-6.7b"

def gen(dataset):
    for data in dataset:
        yield {"text": "<s>" + data[0] + data[1] + "</s>"}

ds = Dataset.from_generator(gen, gen_kwargs={"dataset": json_data})
ds = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)

tokenizer = AutoTokenizer.from_pretrained(galactica_6_7b_path)
tokenizer.add_special_tokens({'pad_token': '<pad>', 'eos_token': '</s>', 'bos_token': '<s>'})

def tokenize(examples):
    model_inputs = tokenizer(examples['text'])
    model_inputs.pop("token_type_ids")
    return model_inputs


ds = ds.map(tokenize, batched=True)
ds = ds.remove_columns(['text'])
ds.set_format(type='torch')

In [3]:
from transformers import DataCollatorForLanguageModeling
from transformers import OPTForCausalLM
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, TaskType
from peft import get_peft_model
import torch


#del model
torch.cuda.empty_cache()

config  = LoraConfig(
    r = 4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    # modules_to_save="null",
    task_type=TaskType.CAUSAL_LM, #type of task to train model on
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
model = OPTForCausalLM.from_pretrained(
                                        galactica_6_7b_path, 
                                        # load_in_8bit=True,
                                        device_map="auto")

model = get_peft_model(model, config)
model.print_trainable_parameters()
model.to(torch.float16)

training_args = TrainingArguments(
    output_dir="./lora_model",
    evaluation_strategy="epoch",
    num_train_epochs=20,
    logging_strategy="steps",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    do_train=True,
    do_eval=True,
    fp16=False,
    bf16=False,
    # optim="paged_adamw_32bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jihuawei/miniconda/envs/pdf-parser/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


trainable params: 4,194,304 || all params: 6,661,554,176 || trainable%: 0.06296284454326113


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1931025874 (jihuawei). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 442.00 MiB (GPU 0; 23.69 GiB total capacity; 21.59 GiB already allocated; 266.06 MiB free; 22.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

# 

# Accelerator().local_process_index

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


0